<a href="https://colab.research.google.com/github/Ebratul/Python/blob/main/NLP/NLP1_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

Skipping, found downloaded files in "./imdb-dataset-of-50k-movie-reviews" (use force=True to force download)


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df = df.iloc[:15000]

In [6]:
df['review'][5]

'Probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. It just never gets old, despite my having seen it some 15 or more times in the last 25 years. Paul Lukas\' performance brings tears to my eyes, and Bette Davis, in one of her very few truly sympathetic roles, is a delight. The kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. And the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. If I had a dozen thumbs, they\'d all be "up" for this movie.'

In [7]:
df['sentiment'].value_counts()

,count
sentiment,
negative,7609
positive,7391


In [8]:
df.isnull().sum()

,0
review,0
sentiment,0


In [9]:
df.duplicated().sum()

np.int64(39)

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
#Basic preprocessing
#remove tags
#Lowercase
#remove stopwords

In [13]:
import re
def remove_tags(raw_text):
  clean_text = re.sub(re.compile('<.*?>'),'', raw_text)
  return clean_text

In [14]:
df['review'] = df['review'].apply(remove_tags)

In [15]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
14995,Bobcat Goldthwait should be commended for atte...,negative
14996,"And it's not because since her days on ""Claris...",positive
14997,A traveling couple (Horton and Hamilton)stumbl...,negative
14998,This film is deeply disappointing. Not only th...,negative


In [16]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [17]:
import nltk
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')
sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x:[item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
df['review']

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
14995,bobcat goldthwait commended attempting somethi...
14996,"since days ""clarissa explains all"" bit crush m..."
14997,traveling couple (horton hamilton)stumble onto...
14998,film deeply disappointing. wenders displays li...


In [20]:
df['review'][5]

'probably all-time favorite movie, story selflessness, sacrifice dedication noble cause, preachy boring. never gets old, despite seen 15 times last 25 years. paul lukas\' performance brings tears eyes, bette davis, one truly sympathetic roles, delight. kids are, grandma says, like "dressed-up midgets" children, makes fun watch. mother\'s slow awakening what\'s happening world roof believable startling. dozen thumbs, "up" movie.'

In [21]:
x = df.iloc[:,0:1]
y = df['sentiment']

In [22]:
x

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
14995,bobcat goldthwait commended attempting somethi...
14996,"since days ""clarissa explains all"" bit crush m..."
14997,traveling couple (horton hamilton)stumble onto...
14998,film deeply disappointing. wenders displays li...


In [23]:
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
14995,negative
14996,positive
14997,negative
14998,negative


In [24]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [25]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [28]:
x_train.shape

(11968, 1)

In [29]:
#apply BOW
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
cv = CountVectorizer()

In [31]:
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [32]:
x_train_bow.shape

(11968, 57157)

In [33]:
x_test_bow.shape

(2993, 57157)

In [34]:
from sklearn.naive_bayes import GaussianNB

In [35]:
gnb = GaussianNB()

In [36]:
gnb.fit(x_train_bow, y_train)

GaussianNB()

In [37]:
y_pred = gnb.predict(x_test_bow)
y_pred

array([0, 0, 0, ..., 0, 1, 0])

In [38]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_test, y_pred)

0.6404944871366521

In [39]:
confusion_matrix(y_test, y_pred)

array([[1123,  396],
       [ 680,  794]])

In [40]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_bow, y_train)
y_pred = rf.predict(x_test_bow)
accuracy_score(y_test, y_pred)

0.8596725693284331

In [45]:
cv = CountVectorizer(ngram_range=(1,4), max_features=3000)
x_train_bow = cv.fit_transform(x_train['review']).toarray()
x_test_bow = cv.transform(x_test['review']).toarray()

In [46]:
rf = RandomForestClassifier()
rf.fit(x_train_bow, y_train)
y_pred = rf.predict(x_test_bow)
accuracy_score(y_test, y_pred)

0.8473103909121283

# Using TF -IDF

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [49]:
x_train_tfidf = tfidf.fit_transform(x_train['review']).toarray()
x_test_tfidf = tfidf.transform(x_test['review'])

In [53]:
print(x_test_tfidf)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 307059 stored elements and shape (2993, 57157)>
  Coords	Values
  (0, 897)	0.04607874977843603
  (0, 1297)	0.040838089044781256
  (0, 1571)	0.050278395928476494
  (0, 1634)	0.02468010242159579
  (0, 1946)	0.03964913501013761
  (0, 1970)	0.037825652026238714
  (0, 2022)	0.017138802224797727
  (0, 2120)	0.021041480864957537
  (0, 2139)	0.046458854571271274
  (0, 2143)	0.027146787960778723
  (0, 2490)	0.01604877985195192
  (0, 2856)	0.023908946769841535
  (0, 3014)	0.02815961030450871
  (0, 3313)	0.09670347604948451
  (0, 3346)	0.02465395601484058
  (0, 3458)	0.033240355541690525
  (0, 3469)	0.028040561252765606
  (0, 3474)	0.03257892261280669
  (0, 3738)	0.03559135963134923
  (0, 3809)	0.021020696620249576
  (0, 4186)	0.01650783424633573
  (0, 4708)	0.04480708662772399
  (0, 5035)	0.032527102773799964
  (0, 5084)	0.02660938154447868
  (0, 5121)	0.024083357810203123
  :	:
  (2992, 38346)	0.08362515341913358
  (2992, 39488)	0.07

In [54]:
rf = RandomForestClassifier()
rf.fit(x_train_tfidf, y_train)
y_pred = rf.predict(x_test_tfidf)
accuracy_score(y_test, y_pred)

0.863013698630137